# EFD latency characterization

This notebook shows how to get data from the InfluxDB API to characterize the total latency for a message from the time it is sent by SAL to the time it is written to InfluxDB.

In [ ]:
import matplotlib
%matplotlib widget
import requests
import numpy
import matplotlib.pyplot as plt
import pandas as pd
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient

## Create the client

In [ ]:
client = EfdClient('int_efd')

## Retrieving timestamps for a given topic
The following timestamps are available (the order reflects the actual message flow through the system) 

- **private_sndStamp**: TAI timestamp in seconds added by SAL when the message was created
- **private_kafkaStamp**: TAI timestamp in seconds added by the SAL Kafka Producer right before it is sent to the EFD
- **time**: UTC timestamp in millisecons when the message is written to InfluxDB.

Note that on March 3, the index timestamp changed to be the `private_sndStamp`.
This was done to make the telemetry easier to analyze as the message index time would be when the telemetry was produced, not when it was ingested some time later.

In [ ]:
t1 = Time('2020-02-27T12:00:00', scale='tai')
window = TimeDelta(3600, format='sec', scale='tai')

In [ ]:
df = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", ["private_sndStamp", "private_kafkaStamp"], t1-window, t1)

In [ ]:
df.head()

## Latency and time in seconds

In [ ]:
df['time'] = numpy.array([(el - numpy.datetime64('1970-01-01T00:00:00')) / numpy.timedelta64(1, 's') for el in df.index.values])
# Latency between SAL Kafka Producer and SAL
df['latency1'] = (df['private_kafkaStamp'] - df['private_sndStamp'])

# Latency between InfluxDB and SAL Kafka Producer, take into account the difference between UTC and TAI
df['latency2'] = df['time'] - df['private_kafkaStamp'] + 37

# Total latency, take into account the differece between UTC and TAI
df['latency_total'] = df['time'] - df['private_sndStamp'] + 37


df['time_seconds'] = df['time']-df['time'][0]

## Latency  characterization

In [ ]:
median = df.latency1.median()
quantile99 = df.latency1.quantile(.99)

p = df.plot(x='time_seconds', y='latency1', figsize=(15,4))
p.set_xlabel("Time (s)")
p.set_ylabel("Latency (s)")
_ = p.text(50,df.latency1.max()-0.01,"Median={:.4f}s 99% percentile={:.2f}s".format(median, quantile99))

In [ ]:
median = df.latency2.median()
quantile99 = df.latency2.quantile(.99)

p = df.plot(x='time_seconds', y='latency2', figsize=(15,4))
p.set_xlabel("Time (s)")
p.set_ylabel("Latency (s)")
_ = p.text(50,df.latency2.max()-0.75,"Median={:.4f}s 99% percentile={:.2f}s".format(median, quantile99))

In [ ]:
median = df.latency_total.median()
quantile99 = df.latency_total.quantile(.99)

p = df.plot(x='time_seconds', y='latency_total', figsize=(15,4))
p.set_xlabel("Time (s)")
p.set_ylabel("Latency (s)")
_ = p.text(50,df.latency_total.max()-0.75,"Median={:.4f}s 99% percentile={:.2f}s".format(median, quantile99))